In [2]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import json
import copy

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


import annotation, image_render, hyspec_io, hyspec_ml, batch_process, misc, preprocess

In [8]:
# Paths
hyspec_dir = "/media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/3b_R_rs_GeoRef"
annotation_json = "/media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/M_Annotation/label_classes.json"
annotation_image_dir = "/media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/M_Annotation/All"
classifier_file = "/media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/M_Classification/Classifiers/Sandsund_20210602_SimpleRF"

classification_results_dir = "/media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/M_Classification/Results"

In [3]:
# Read class dictionary
class_dict = annotation.read_hasty_metadata(annotation_json)

In [6]:
# Load classifier from file
classifier = misc.pickle_load_data(classifier_file)

In [7]:
classifier

Pipeline(steps=[('wavelengthselector',
                 WavelengthSelector(wl=array([ 390.86,  392.87,  394.88,  396.89,  398.9 ,  400.91,  402.92,
        404.93,  406.94,  408.96,  410.97,  412.99,  415.  ,  417.02,
        419.04,  421.06,  423.08,  425.1 ,  427.12,  429.14,  431.17,
        433.19,  435.22,  437.24,  439.27,  441.3 ,  443.33,  445.36,
        447.39,  449.42,  451.45,  453.49,  455.52,  457.56,  459.59,
        461.63,  463.67,  465.71,  467.75,  469.79,  471.83,  4...
        956.09,  958.33,  960.58,  962.82,  965.07,  967.32,  969.56,
        971.81,  974.06,  976.31,  978.57,  980.82,  983.07,  985.33,
        987.58,  989.84,  992.1 ,  994.36,  996.62,  998.88, 1001.14,
       1003.4 , 1005.66, 1007.92, 1010.19, 1012.45, 1014.72, 1016.99,
       1019.26, 1021.52, 1023.79, 1026.07, 1028.34, 1030.61]),
                                    wl_max=900, wl_min=400)),
                ('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=7)),


In [4]:
# Get all hyperspectral files
hyspec_files = misc.file_pattern_search(hyspec_dir,'*.hdr')

In [5]:
hyspec_files

['/media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/3b_R_rs_GeoRef/Sandsund_May2021_Pika_L_10-Radiance From Raw Data-Reflectance from Radiance Data and Downwelling Irradiance Spectrum-Georectify Airborne Datacube.bip.hdr',
 '/media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/3b_R_rs_GeoRef/Sandsund_May2021_Pika_L_11-Radiance From Raw Data-Reflectance from Radiance Data and Downwelling Irradiance Spectrum-Georectify Airborne Datacube.bip.hdr',
 '/media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/3b_R_rs_GeoRef/Sandsund_May2021_Pika_L_12-Radiance From Raw Data-Reflectance from Radiance Data and Downwelling Irradiance Spectrum-Georectify Airborne Datacube.bip.hdr',
 '/media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/3b_R_rs_GeoRef/Sandsund_May2021_Pika_L_13-Radiance From Raw Data-Reflectance from Radiance Data and Downwelling Irradiance Spectrum-Georectify Airborne Datacube.bip.hdr',
 '/media/mha114/Massimal/Bodo_Sandsund/Hyperspec

In [9]:
classification_files = misc.build_newdir_filepath(hyspec_files,classification_results_dir)

In [11]:
for (input_file,output_file) in zip (hyspec_files, classification_files):
    
    # Load image
    print('Loading file ' + input_file)
    (im,wl,rgb_ind,metadata) = hyspec_io.load_envi_image(input_file)
    
    # Apply classifier
    y_pred_im = hyspec_ml.apply_classifier_to_image(classifier,im)
    
    # Save classification result
    print('Saving result to ' + output_file)
    hyspec_ml.save_classification_as_envi(y_pred_im,metadata['map info'],output_file)

Loading file /media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/3b_R_rs_GeoRef/Sandsund_May2021_Pika_L_10-Radiance From Raw Data-Reflectance from Radiance Data and Downwelling Irradiance Spectrum-Georectify Airborne Datacube.bip.hdr
Saving result to /media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/M_Classification/Results/Sandsund_May2021_Pika_L_10-Radiance From Raw Data-Reflectance from Radiance Data and Downwelling Irradiance Spectrum-Georectify Airborne Datacube.bip.hdr
Loading file /media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/3b_R_rs_GeoRef/Sandsund_May2021_Pika_L_11-Radiance From Raw Data-Reflectance from Radiance Data and Downwelling Irradiance Spectrum-Georectify Airborne Datacube.bip.hdr
Saving result to /media/mha114/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/M_Classification/Results/Sandsund_May2021_Pika_L_11-Radiance From Raw Data-Reflectance from Radiance Data and Downwelling Irradiance Spectrum-Georectify Airborne D